# Query Shared MotherDuck Database

## Overview

This notebook demonstrates how to take a shared MotherDuck database and query it using python.

In [ ]:
import os
import tomllib
import duckdb

# Load the TOML file
with open("config.toml", "rb") as file:
    config = tomllib.load(file)

# Extract the MD_SERVICE_TOKEN from the loaded config
md_service_token = config['mother-duck']['MD_SERVICE_TOKEN']

os.environ['motherduck_token'] = md_service_token

con = duckdb.connect(f'md:?motherduck_token={md_service_token}') 

In [ ]:
md_shared_db = config['mother-duck']['MD_SHARED_DB']

try:
    con.sql(f"ATTACH '{md_shared_db}' AS md_shared_db")
except Exception as e:
    if "already attached" in str(e):
        print("Already attached")
    else:
        # Handle other exceptions if needed
        print("An error occurred:", e)

Already attached


In [ ]:
import polars as pl
import pandas as pd
import ibis

#con.sql("USE voc_omop54_shared")

# Polars
#df = con.sql("SELECT * FROM concept").pl()

# Pandas
#df = con.sql("SELECT * FROM concept").df()

# Ibis
duckdb_con = ibis.connect(f'duckdb://md:voc_omop54_shared')

In [ ]:
ibis.options.interactive = True
duckdb_con.list_tables()

['concept',
 'concept_ancestor',
 'concept_class',
 'concept_recommended',
 'concept_relationship',
 'concept_synonym',
 'vocabulary']

In [ ]:
table_of_interest = 'concept'



df_concept = duckdb_con.table(f'{table_of_interest}')


df_filtered_standard_concepts_null = df_concept.filter(df_concept.standard_concept.isnull())
df_filtered_standard_concepts_null_count = df_filtered_standard_concepts_null.count()

print(f'{table_of_interest} has a total of {df_filtered_standard_concepts_null_count} non-standard concepts.')


df_filtered_standard_concepts_notnull = df_concept.filter(~df_concept.standard_concept.isnull())
df_filtered_standard_concepts_notnull_count = df_filtered_standard_concepts_notnull.count()

print(f'{table_of_interest} has a total of {df_filtered_standard_concepts_notnull_count} standard concepts.')



df_filtered_standard_concepts_notnull.head(5)

concept has a total of 5166471 non-standard concepts.


concept has a total of 4274904 standard concepts.


┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ concept_id ┃ concept_name                   ┃ domain_id ┃ vocabulary_id ┃ concept_class_id ┃ standard_concept ┃ concept_code ┃ valid_start_date ┃ valid_end_date ┃ invalid_reason ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ int32      │ string                         │ string    │ string        │ string           │ string           │ string       │ date             │ date           │ string         │
├────────────┼────────────────────────────────┼───────────┼───────────────┼──────────────────┼──────────────────┼──────────────┼──────────────────┼────────────────┼────────────────┤
│   45053964 │ CETAPHIL MOISTURIZING CREAM    │ Device    │ NDC           │ Device           │ S                │ 54868012900  │ 2013-08-05       │ 2099-12-31     │ NULL           │
│   45054039 │ CHEMSTRIP 9 TEST STRIPS        │ Device    │ NDC           │ Device           │ S                │ 54868242000  │ 2013-08-05       │ 2099-12-31     │ NULL           │
│   45054831 │ OPSITE DRESSING 5 1/2" X 4"    │ Device    │ NDC           │ Device           │ S                │ 55860004975  │ 2013-08-05       │ 2099-12-31     │ NULL           │
│   45054909 │ NEEDLE FREE SYRINGE KIT C      │ Device    │ NDC           │ Device           │ S                │ 55948022610  │ 2013-08-05       │ 2099-12-31     │ NULL           │
│   45054949 │ TRUEPLUS SUPER THIN 28G LANCET │ Device    │ NDC           │ Device           │ S                │ 56151024221  │ 2013-08-05       │ 2099-12-31     │ NULL           │
└────────────┴────────────────────────────────┴───────────┴───────────────┴──────────────────┴──────────────────┴──────────────┴──────────────────┴────────────────┴────────────────┘

In [ ]:
df_concept = df_concept['concept_id', 'concept_name', 'domain_id', 'vocabulary_id', 'concept_class_id', 'standard_concept', 'concept_code'].head(2000)


In [ ]:
df1 = df_concept.to_pandas()
#df1.rename(columns={'concept_id': 'unique_id'}, inplace=True)
df1

,unique_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code
0,45053734,Etodolac 300 MG Oral Capsule [Lodine],Drug,NDC,11-digit NDC,None,54569326401
1,45053735,Cephalexin 500 MG Oral Capsule,Drug,NDC,11-digit NDC,None,54569332406
2,45053736,Amoxicillin 500 MG Oral Capsule,Drug,NDC,11-digit NDC,None,54569333501
3,45053737,12 HR Hyoscyamine Sulfate 0.375 MG Extended Re...,Drug,NDC,11-digit NDC,None,54569349600
4,45053738,Chloroprocaine hydrochloride 10 MG/ML Injectab...,Drug,NDC,11-digit NDC,None,54569350000
...,...,...,...,...,...,...,...
1995,45055729,Neomycin Sulfate 500 MG Oral Tablet,Drug,NDC,11-digit NDC,None,58016076777
1996,45055730,Neomycin Sulfate 500 MG Oral Tablet,Drug,NDC,11-digit NDC,None,58016076781
1997,45055731,Neomycin Sulfate 500 MG Oral Tablet,Drug,NDC,11-digit NDC,None,58016076789
1998,45055732,Divalproex Sodium 500 MG Enteric Coated Tablet...,Drug,NDC,11-digit NDC,None,58016077407
